### Imports

In [1]:
import http.client
import urllib.request
import json
import datetime
import time
import csv

import plotly.plotly as py
import plotly.graph_objs as go

# Bittrex

### API Keys

In [2]:
API_KEY = 'xxx'
API_SECRET = 'xxx'

### Bittrex API Wrapper

In [3]:
public_methods = ['getmarkets', 'getcurrencies', 'getticker', 'getmarketsummaries', 'getmarketsummary', 'getorderbook', 'getmarkethistory']
generic_methods = ['getmarkets', 'getcurrencies', 'getmarketsummaries']
specific_methods = ['getticker', 'getmarketsummary', 'getorderbook', 'getmarkethistory']

class bittrex:
    
    # CONSTRUCTORS
    
    def __init__(self, key, secret):
        self.key = key
        self.secret = secret
        
    def __init__(self):
        self.key = None
        self.secret = None
    
    # PUBLIC API METHODS
    
    public_url = 'https://bittrex.com/api/v1.1/public/'
    
    def getMarkets(self):
        url = public_url + 'getmarkets'
        r = urllib.request.urlopen(url)
        if json.loads(r.read())['sucess']:
            result = json.loads(r.read())['result']
            markets = []
            for market in result:
                if json.loads(market)['active']:
                    markets.append((json.loads(market)['BaseCurrency'], json.loads(market)['MarketCurrency']))
            return markets
        else:
            raise Exception('Unsucessful Request')
            return None
    
    def getCurrencies(self):
        url = public_url + 'getcurrencies'
        r = urllib.request.urlopen(url)
        if json.loads(r.read())['sucess']:
            result = json.loads(r.read())['result']
            currencies = []
            for currency in result:
                currencies.append(json.loads(currency)['Currency'])
            return currencies
        else:
            raise Exception('Unsucessful Request')
            return None
    
    def getTicker(self, base, market):
        url = public_url + 'getticker?market=' + base + '-' + market
        r = urllib.request.urlopen(url)
        if json.loads(r.read())['sucess']:
            result = json.loads(json.loads(r.read())['result'])
            return result['Bid'], result['Ask'], result['Last']
        else:
            raise Exception('Unsucessful Request')
            return None
    
    def getMarketSummary(self, base, market):
        url = public_url + 'getmarketsummary?market=' + base + '-' + market
        r = urllib.request.urlopen(url)
        if json.loads(r.read())['sucess']:
            result = json.loads(json.loads(r.read())['result'])
            return result['High'], result['Low'], result['BaseVolume']
        else:
            raise Exception('Unsucessful Request')
            return None
    
    def getOrderBook(self, base, market, order):
        url = public_url + 'getmarketsummary?market=' + base + '-' + market + '&type=' + order
        r = urllib.request.urlopen(url)
        if json.loads(r.read())['sucess']:
            result = json.loads(r.read())['result']
            buys = []
            for buy in json.loads(result)['buy']:
                buys.append((json.loads(buy)['Quantity'], json.loads(buy)['Rate']))
            sells = []
            for sell in json.loads(result)['sell']:
                sells.append((json.loads(buy)['Quantity'], json.loads(buy)['Rate']))
            return buys, sells
        else:
            raise Exception('Unsucessful Request')
            return None
    
    def getOpenOrders(self, base, market):
        url = public_url + 'getopenorders?market=' + base + '-' + market
        r = urllib.request.urlopen(url)
        if json.loads(r.read())['sucess']:
            result = json.loads(r.read())['result']
            limit_buys = []
            limit_sells = []
            for order in result:
                order = json.loads(order)
                if order['OrderType'] is 'LIMIT_BUY':
                    limit_buys.append(order['Quantity'], order['Price'])
                elif json.loads(order)['OrderType'] is 'LIMIT_SELL':
                    limit_sells.append(order['Quantity'], order['Price'])
            return limit_buys, limit_sells
        else:
            raise Exception('Unsucessful Request')
            return None
    
    # MARKET API METHODS
    
    
    
    # ACCOUNT API METHODS
    
    account_url = 'https://bittrex.com/api/v1.1/account/'
    api_url = '?apikey=' + self.key
    
    def getBalances(self):
        url = account_url + 'getbalances' + api_url
        r = urllib.request.urlopen(url)
        if json.loads(r.read())['sucess']:
            result = json.loads(r.read())['result']
            balances = []
            for currency in result:
                if json.loads(balance)['Balance'] > 0:
                    currency = json.loads(currency)
                    balances.append((currency['Currency'], currency['Balance'], currency['Avaliable']))
            return balances
        else:
            raise Exception('Unsucessful Request')
            return None
    
    def getBalance(self, currency):
        url = account_url + 'getbalance' + api_url + '&currency=' + currency
        r = urllib.request.urlopen(url)
        if json.loads(r.read())['sucess']:
            result = json.loads(r.read())['result']
            return (json.loads(result)['Balance'], json.loads(result)['Avaliable'])
        else:
            raise Exception('Unsucessful Request')
            return None
    
    def getOrder(self, uuid):
        url = account_url + 'getorder' + api_url + '&uuid=' + uuid
        r = urllib.request.urlopen(url)
        if json.loads(r.read())['sucess']:
            result = json.loads(json.loads(r.read())['result'])
            return (result['Exchange'], result['Type'], result['Price'], result['Quantity'], result['QuanitiyRemaining'], result['Opened'])
        else:
            raise Exception('Unsucessful Request')
            return None
    
    def getOrderHistory(self, base, market):
        url = account_url + 'getorderhistory' + api_url + '?market=' + base + '-' + market
        r = urllib.request.urlopen(url)
        if json.loads(r.read())['sucess']:
            result = json.loads(json.loads(r.read())['result'])
            orders = []
            for order in result:
                orders.append(result['OrderType'], result['Price'], result['Quantity'], result['QuanitiyRemaining'], result['TimeStamp'])
        else:
            raise Exception('Unsucessful Request')
            return None
    
    
    
    def market_stream(self, currency, market, interval_min, frequency_sec):
        iterations = int(interval_min * 60 / frequency_sec)
        price_data = []
        time_log = []
        day = datetime.datetime.now().day
        for i in range(0, iterations):
            price_data.append(round(public_query('getticker', currency, market)['Last'], 10))
            time_log.append((datetime.datetime.now().minute, datetime.datetime.now().second))
            time.sleep(frequency_sec)
        return price_data, time_log, day

# Poloniex

### Poloniex Wrapper

In [6]:
def poloniex_date_to_UNIX(date):
    # date in format yr-mo-dy hr:mi:se
    date = str(date)
    year = int(date[:4])
    month = int(date[5:-12])
    day = int(date[8:-9])
    hour = int(date[11:-6])
    minute = int(date[14:-3])
    second = int(date[17:])
    dt = datetime.datetime(year, month, day, hour, minute, second, 0)
    UNIX = time.mktime(dt.timetuple())
    UNIX = str(UNIX)[:-2]
    return UNIX

def return_trade_history_by_time (market, currency, start_date, end_date):
        r = urllib.request.urlopen('https://poloniex.com/public?command=returnTradeHistory&currencyPair=' + market + '_' + currency + '&start=' + str(poloniex_date_to_UNIX(start_date)) + '&end=' + str(poloniex_date_to_UNIX(end_date)))
        return json.loads(r.read())

def get_trade_history(market, currency, changing_date, end_date, prev_date = None, writer = None):
    if writer is None:
        keys = {'globalTradeID': 15429515, 'tradeID': 1013613, 'date': '2016-02-10 04:15:21', 'type': 'sell', 'rate': '0.00795108', 'amount': '0.81580000', 'total': '0.00648649'}.keys()
        f = open(market + '_' + currency + '_dates.csv', 'w+')
        writer = csv.DictWriter(f, keys)
        writer.writeheader()
    if changing_date == end_date or prev_date == end_date:
        return None
    else:
        trades = return_trade_history_by_time(market, currency, changing_date, end_date)
        if (len(trades) > 1):   
            writer.writerows(trades)
            recent_date = str(trades[-1]['date'])
            get_trade_history(market, currency, recent_date, end_date, changing_date, writer)
        else:
            return None

### Getting Trade Data

In [7]:
get_trade_history('BTC', 'STRAT', '2017-12-22 01:01:01', '2018-01-22 01:01:01')

KeyboardInterrupt: 

### Graphing Trade Data

In [10]:
btc_strat_trades =[]
with open('BTC_STRAT_dates.csv') as csvfile:
        r = csv.DictReader(csvfile)
        for row in r:
            if row[0] is None:
                continue
            btc_strat_trades.append((row['rate'], row['rate']))

AttributeError: 'DictReader' object has no attribute 'rows'

In [8]:
py.iplot(btc_strat_trades)

PlotlyError: Invalid 'figure_or_data' argument. Plotly will not be able to properly parse the resulting JSON. If you want to send this 'figure_or_data' to Plotly anyway (not recommended), you can set 'validate=False' as a plot option.
Here's why you're seeing this error:

Invalid entry found in 'data' at index, '0'

Path To Error: ['data'][0]

Valid items for 'data' at path ['data'] under parents ['figure']:
    ['Area', 'Bar', 'Box', 'Candlestick', 'Carpet', 'Choropleth',
    'Contour', 'Contourcarpet', 'Heatmap', 'Heatmapgl', 'Histogram',
    'Histogram2d', 'Histogram2dcontour', 'Mesh3d', 'Ohlc', 'Parcoords',
    'Pie', 'Pointcloud', 'Sankey', 'Scatter', 'Scatter3d', 'Scattercarpet',
    'Scattergeo', 'Scattergl', 'Scattermapbox', 'Scatterternary',
    'Surface', 'Table']

Entry should subclass dict.